In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
DEFAULT_CRS = "EPSG:4326"

In [3]:

hex_sp = gpd.read_file("data/shapeFiles/GRIDS_H3_SP_RES10") 


# Transform the CRS to EPSG:31983
hex_sp = hex_sp.to_crs(DEFAULT_CRS)
hex_sp.head()

,id_hex,geometry
0,8aa8100c3257fff,"POLYGON ((-46.63592 -23.53032, -46.63664 -23.5..."
1,8aa8100d2447fff,"POLYGON ((-46.53650 -23.53972, -46.53722 -23.5..."
2,8aa8100e1637fff,"POLYGON ((-46.66886 -23.59437, -46.66958 -23.5..."
3,8aa8100f0827fff,"POLYGON ((-46.59115 -23.61353, -46.59187 -23.6..."
4,8aa81010ec07fff,"POLYGON ((-46.65231 -23.83331, -46.65303 -23.8..."


In [4]:
# Read the shapefile
quadras = gpd.read_file("data/shapeFiles/SIRGAS_SHP_quadraMDSF")

# Set the CRS to EPSG:31983 (SIRGAS 2000 / UTM zone 23S)
quadras = quadras.set_crs(epsg=31983)

In [5]:
quadras_centroids = quadras.copy()

# Step 1: Calculate centroids
quadras_centroids['geometry'] = quadras_centroids.geometry.centroid

quadras_centroids.head()

,qd_id_orig,qd_setor,qd_fiscal,qd_tipo,qd_subqua,qd_area,qd_situac,qd_id,qd_tx_tipo,geometry
0,10346990,153,029,F,001,4859.1528512042,1,138007,FISCAL,POINT (347264.627 7387208.177)
1,10117715,139,181,F,001,5347.62152613216,1,138008,FISCAL,POINT (353317.502 7398534.699)
2,10119816,192,150,F,001,6928.73697365148,1,138009,FISCAL,POINT (357968.765 7398273.689)
3,8895416,008,030,F,001,10531.7999187608,1,138010,FISCAL,POINT (332572.522 7396857.792)
4,10273851,138,217,F,001,5638.25762262573,1,138011,FISCAL,POINT (354492.570 7396537.947)


In [6]:
if quadras_centroids.crs != DEFAULT_CRS:
    quadras_centroids = quadras_centroids.to_crs(DEFAULT_CRS)
if hex_sp.crs != DEFAULT_CRS:
    hex_sp = hex_sp.to_crs(DEFAULT_CRS)

In [7]:
# Step 2: Perform spatial intersection with `hex_sp`
intersection_result = quadras_centroids.overlay(hex_sp, how='intersection')
hex_sp
# Step 3: Drop the geometry column
hex_centroides_quadras = intersection_result.drop(columns='geometry')

hex_centroides_quadras.head()

,qd_id_orig,qd_setor,qd_fiscal,qd_tipo,qd_subqua,qd_area,qd_situac,qd_id,qd_tx_tipo,id_hex
0,10346990,153,029,F,001,4859.1528512042,1,138007,FISCAL,8aa81008ed27fff
1,10117715,139,181,F,001,5347.62152613216,1,138008,FISCAL,8aa810733177fff
2,10119816,192,150,F,001,6928.73697365148,1,138009,FISCAL,8aa810449117fff
3,8895416,008,030,F,001,10531.7999187608,1,138010,FISCAL,8aa8100cec9ffff
4,10273851,138,217,F,001,5638.25762262573,1,138011,FISCAL,8aa810730417fff


In [10]:
hex_centroides_quadras = hex_centroides_quadras[['id_hex', 'qd_setor']]

In [11]:
iptu = pd.read_csv(
    "data/IPTU_2024.csv",
    sep=";",
    encoding='latin-1')


/var/folders/kf/12t4yv8j7pg0z8zqbypmj9440000gn/T/ipykernel_47065/274605220.py:1: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  iptu = pd.read_csv(


In [12]:
iptu.head()

,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NUMERO DA NL,DATA DO CADASTRAMENTO,NUMERO DO CONDOMINIO,CODLOG DO IMOVEL,NOME DE LOGRADOURO DO IMOVEL,NUMERO DO IMOVEL,COMPLEMENTO DO IMOVEL,BAIRRO DO IMOVEL,...,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,TIPO DE TERRENO,FATOR DE OBSOLESCENCIA,ANO DE INICIO DA VIDA DO CONTRIBUINTE,MES DE INICIO DA VIDA DO CONTRIBUINTE,FASE DO CONTRIBUINTE
0,0010030001-4,2024,1,01/01/24,00-0,03812-1,R S CAETANO,13.0,NaN,SANTA EFIGENIA,...,1924.0,1,13.0,Loja,Comercial horizontal - padrão B,De esquina,0.20,1963,1.0,0.0
1,0010030002-2,2024,1,01/01/24,00-0,03812-1,R S CAETANO,19.0,NaN,SANTA EFIGENIA,...,1944.0,1,6.0,Loja,Comercial horizontal - padrão B,Normal,0.20,1963,1.0,0.0
2,0010030003-0,2024,1,01/01/24,00-0,03812-1,R S CAETANO,27.0,NaN,SANTA EFIGENIA,...,1965.0,2,7.85,Loja,Comercial horizontal - padrão B,Normal,0.22,1963,1.0,0.0
3,0010030004-9,2024,1,01/01/24,00-0,03812-1,R S CAETANO,33.0,NaN,NaN,...,1944.0,1,6.05,Loja,Comercial horizontal - padrão B,Normal,0.20,1963,1.0,0.0
4,0010030005-7,2024,1,01/01/24,00-0,03812-1,R S CAETANO,39.0,NaN,NaN,...,2004.0,2,6.7,Loja,Comercial horizontal - padrão B,Normal,0.82,1963,1.0,0.0


In [13]:

# Define the list of valid categories for filtering
valid_categories = [
    'Apartamento em condomínio',  # Encoding adjustment
    'Flat residencial em condomínio',
    'Não residencial',
    'Prédio de apartamento, não em condomínio, de uso exclusivamente residencial',
    'Prédio de apartamento, não em condomínio, de uso misto (apartamentos e escritórios e/ou consultórios), com ou sem loja (predominância residencial)',
    'Residência',
    'Residência e outro uso (predominância residencial)'
]

# Filter the DataFrame based on 'TIPO.DE.USO.DO.IMOVEL' values
iptu_fil = iptu[iptu['TIPO DE USO DO IMOVEL'].isin(valid_categories)].copy()

# Create new columns based on substrings from 'NUMERO.DO.CONTRIBUINTE'
iptu_fil['qd_setor'] = iptu_fil['NUMERO DO CONTRIBUINTE'].str[:3]
iptu_fil['qd_fiscal'] = iptu_fil['NUMERO DO CONTRIBUINTE'].str[3:6]
iptu_fil['cd_lote'] = iptu_fil['NUMERO DO CONTRIBUINTE'].str[6:10]

# Select only the new columns
iptu_fil = iptu_fil[['qd_setor', 'qd_fiscal', 'cd_lote']]

del iptu


In [14]:
iptu_fil.head()

,qd_setor,qd_fiscal,cd_lote
5,001,003,0006
16,001,003,0023
20,001,003,0028
21,001,003,0029
22,001,003,0030


## Agrupar 

In [15]:
chunks = np.array_split(iptu_fil, 10)  # Split into 10 smaller chunks
result = pd.DataFrame()

# Step 1: Perform a left join 
for chunk in chunks:
    chunk_result = chunk.join(hex_centroides_quadras.set_index("qd_setor"), how='left', on='qd_setor')
    result = pd.concat([result, chunk_result], ignore_index=True)

#hex_lancamentos_iptu = iptu_fil.join(
#        ,
#        how='left', on='qd_setor')  # Assuming 'qd_setor' is the join key, adjust as needed
# Step 2: Group by 'fid' and count the occurrences




/opt/anaconda3/envs/scriptsMestrado/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [16]:
result.head()

,qd_setor,qd_fiscal,cd_lote,id_hex
0,001,003,0006,8aa8100c30c7fff
1,001,003,0006,8aa8100c380ffff
2,001,003,0006,8aa8100c394ffff
3,001,003,0006,8aa8100c3b27fff
4,001,003,0006,8aa8100c3b8ffff


In [17]:
hex_lancamentos_iptu = result.groupby('id_hex').size().reset_index(name='n_lancamentos')

In [18]:
# Step 1: Perform a right join
hex_iptu = pd.merge(hex_lancamentos_iptu, hex_sp, how='right', on='id_hex')  # Adjust 'fid' if necessary

# Step 2: Convert the result to a GeoDataFrame
hex_iptu = gpd.GeoDataFrame(hex_iptu)

# Step 3: Ensure geometries are valid
hex_iptu['geometry'] = hex_iptu['geometry'].apply(lambda geom: geom if geom.is_valid else geom.buffer(0))

# Alternatively, you can use:
# hex_iptu = hex_iptu[hex_iptu.is_valid].copy()

# (Optional) If there are no geometries to check, ensure the geometry column exists
if 'geometry' not in hex_iptu.columns:
    hex_iptu['geometry'] = None  # or handle appropriately

# Now hex_iptu is ready for use


In [19]:
hex_iptu.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [20]:

# Write the GeoDataFrame to a shapefile
hex_iptu.to_file("data/WGS84_hex_iptu.gpkg", driver="GPKG")
